#사전 세팅


In [1]:
import nltk
from nltk.corpus import wordnet
from sklearn.utils import resample
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# nltk 데이터 다운로드 (한 번만 실행 필요)
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
# 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train.csv')

#함수들


In [3]:
#문장순서 랜덤으로 섞기
def shuffle_sentences(text):
    sentences = text.split('.')  # 문장을 마침표로 분리
    random.shuffle(sentences)     # 문장 순서를 무작위로 섞기
    return '.'.join(sentences)


In [4]:
#유사도 비슷한 것 끼리 합치기
def combine_articles(df):
    combined_data = []

    for category in df['분류'].unique():
        category_data = df[df['분류'] == category].reset_index(drop=True)

        # 카테고리 데이터가 10,000개 이상일 경우에만 진행
        if len(category_data) > 30000:

            # 제목과 키워드를 합친 텍스트 데이터 생성
            category_data['combined_text'] = category_data['제목'] + " " + category_data['키워드']

            # TF-IDF 벡터화
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(category_data['combined_text'])

            # 코사인 유사도 계산
            cosine_sim = cosine_similarity(tfidf_matrix)

            # 유사한 기사 2개씩 묶기
            used_indices = set()
            for i in range(len(category_data)):
                if i in used_indices:
                    continue
                most_similar_idx = np.argsort(cosine_sim[i])[-2]  # 자기 자신을 제외한 가장 유사한 기사 선택
                if most_similar_idx not in used_indices:
                    combined_title = category_data['제목'][i] + " " + category_data['제목'][most_similar_idx]
                    combined_keywords = category_data['키워드'][i] + "," + category_data['키워드'][most_similar_idx]
                    combined_entry = {
                        '분류': category,
                        '제목': combined_title,
                        '키워드': combined_keywords
                    }
                    combined_data.append(combined_entry)
                    used_indices.add(i)
                    used_indices.add(most_similar_idx)

    return pd.DataFrame(combined_data)

# 데이터프레임에서 유사한 기사 2개씩 묶기



In [5]:
# # 기사와 키워드 2개씩 합치기
# def combine_articles(df):
#     combined_data = []

#     for category in df['분류'].unique():
#         category_data = df[df['분류'] == category].reset_index(drop=True)

#         # 5개씩 묶어서 합치기
#         for i in range(0, len(category_data), 2):
#             combined_title = " ".join(category_data['제목'][i:i+2])
#             combined_keywords = ",".join(category_data['키워드'][i:i+2])
#             combined_entry = {
#                 '분류': category,
#                 '제목': combined_title,
#                 '키워드': combined_keywords
#             }
#             combined_data.append(combined_entry)

#     return pd.DataFrame(combined_data)

# # 카테고리별로 5개씩 합친 데이터프레임 생성

In [6]:
# #데이터 증강
# def augment_data(df, target_count=20000):
#     augmented_df = df.copy()

#     # 각 카테고리별로 데이터 증강
#     for category in df['분류'].unique():
#         category_data = augmented_df[augmented_df['분류'] == category]
#         current_count = len(category_data)

#         # 200개 미만이면 증강
#         if current_count < target_count:
#             additional_samples_needed = target_count - current_count
#             augmented_entries = resample(category_data, replace=True, n_samples=additional_samples_needed, random_state=42)

#             augmented_entries['제목'] = augmented_entries['제목'].apply(shuffle_sentences)
#             augmented_entries['키워드'] = augmented_entries['키워드'].apply(shuffle_sentences)

#             augmented_df = pd.concat([augmented_df, augmented_entries], ignore_index=True)

#     return augmented_df


In [7]:
import pandas as pd
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
import random

# EDA 증강을 위한 간단한 함수들
def shuffle_sentences(text):
    words = text.split()
    random.shuffle(words)
    return ' '.join(words)

def synonym_replacement(text, alpha=0.1):
    # 간단한 동의어 치환 예시
    words = text.split()
    num_words_to_replace = int(len(words) * alpha)

    for _ in range(num_words_to_replace):
        idx_to_replace = random.randint(0, len(words) - 1)
        words[idx_to_replace] = random.choice(words)  # 실제 동의어 치환 대신 랜덤 단어로 대체

    return ' '.join(words)

def eda_augment(text, alpha_synonym=0.1):
    text = shuffle_sentences(text)
    text = synonym_replacement(text, alpha_synonym)
    return text

# 데이터 증강 함수
def augment_data(df, target_count=20000):
    augmented_df = df.copy()
    smote = SMOTE(random_state=42)

    # 각 카테고리별로 데이터 증강
    for category in df['분류'].unique():
        category_data = augmented_df[augmented_df['분류'] == category]
        current_count = len(category_data)

        # 데이터가 target_count보다 적은 경우 EDA와 SMOTE를 적용
        if current_count < target_count:
            additional_samples_needed = target_count - current_count
            augmented_entries = resample(category_data, replace=True, n_samples=additional_samples_needed, random_state=42)

            # EDA 증강
            augmented_entries['제목'] = augmented_entries['제목'].apply(eda_augment)
            augmented_entries['키워드'] = augmented_entries['키워드'].apply(eda_augment)

            augmented_df = pd.concat([augmented_df, augmented_entries], ignore_index=True)

    # SMOTE 증강 (적절한 텍스트 벡터화 과정 필요)
    X = augmented_df[['제목', '키워드']]  # 텍스트 데이터를 벡터화해야 함
    y = augmented_df['분류']

    # SMOTE를 적용하려면 텍스트를 벡터로 변환해야 함 (예: TF-IDF)
    # 이 부분은 텍스트 임베딩을 위한 적절한 방법을 추가해야 함
    # vectorizer = TfidfVectorizer()  # 예시
    # X_vectorized = vectorizer.fit_transform(X['제목'])  # '제목'을 벡터화

    # SMOTE 적용
    # X_smote, y_smote = smote.fit_resample(X_vectorized, y)

    # 이후 벡터화된 텍스트를 다시 복원하는 로직이 필요
    return augmented_df

# 예시 데이터프레임에서 증강 실행
# augmented_df = augment_data(original_df)


In [8]:
# 카테고리별로 10000개가 넘는 경우 5000개로 줄이기
def reduce_large_categories(df, max_count=10000, threshold=30000):
    reduced_df = pd.DataFrame()

    for category in df['분류'].unique():
        category_data = df[df['분류'] == category]
        if len(category_data) > threshold:
            # 10000개가 넘는 경우, 랜덤으로 5000개 선택
            category_data = resample(category_data, replace=False, n_samples=max_count, random_state=42)
        reduced_df = pd.concat([reduced_df, category_data], ignore_index=True)

    return reduced_df

#실행


In [10]:
# combined_df = combine_articles(train)

# 각 카테고리 당 최소 ??개로 증강 (문장 순서 변경 포함)
# augmented_df = augment_data(combined_df, target_count=50)
augmented_df = augment_data(train, target_count=20000)
# 카테고리 별로 10000개가 넘는 경우 ??개로 줄이기
#final_df = reduce_large_categories(augmented_df, max_count=2000, threshold=10000)

final_df=augmented_df

# 줄인 데이터 확인
print(final_df['분류'].value_counts())
# 증강된 데이터 저장 (선택 사항)
final_df.to_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train_augmented_combined10.csv', index=False)


분류
지역               26950
문화:전시_공연         20000
사회:날씨            20000
문화:방송_연예         20000
IT_과학:IT_과학일반    20000
경제:반도체           20000
경제:자원            20000
문화:종교            20000
경제:자동차           20000
경제:무역            20000
IT_과학:콘텐츠        20000
경제:증권_증시         20000
IT_과학:과학         20000
경제:외환            20000
문화:요리_여행         20000
정치:청와대           20000
문화:출판            20000
문화:미술_건축         20000
문화:음악            20000
스포츠:농구_배구        20000
문화:생활            20000
정치:북한            20000
스포츠:야구           20000
문화:학술_문화재        20000
IT_과학:보안         20000
문화:영화            20000
스포츠:월드컵          20000
사회:여성            20000
IT_과학:모바일        20000
경제:유통            20000
IT_과학:인터넷_SNS    20000
국제               20000
정치:선거            20000
경제:금융_재테크        20000
사회:의료_건강         20000
정치:행정_자치         20000
정치:국회_정당         20000
스포츠:축구           20000
경제:서비스_쇼핑        20000
경제:부동산           20000
사회:교육_시험         20000
사회:미디어           20000
사회:장애인           20000
스포츠:골프  

In [11]:
final_df.head(30)

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."
5,TRAIN_00005,문화:전시_공연,"소강석 목사 ""한미 참전용사 끝까지 찾아뵐 것""","소강석,목사,한미,참전용사,한미,참전,용사,초청,행사,방한,초청,종료,한미,참전,용..."
6,TRAIN_00006,정치:선거,"[지방선거 D-360] 전 현직 공직출신 후보군, 행정경험 무기로 '단체장' 노린다","현직,공직,출신,후보군,행정,경험,무기,단체장,지방,선거,1년,현직,공직,출신,인사..."
7,TRAIN_00007,경제:금융_재테크,"용인시, '자동차세 연납제도'로 9.15% 세액공제","용인시,자동차세,연납,제도,9.15%,세액,공제,용인시청,용인시,최대,10%,자동차..."
8,TRAIN_00008,지역,울진 '원자력수소 국가산단' 행정절차 '속속'...기본협약 체결,"국가산단,울진,원자,력수소,체결,행정,절차,기본,협약,남효선,경북,울진군,미래,먹거..."
9,TRAIN_00009,지역,"용인시, ‘용인-화성 광역 버스정보시스템 구축사업’ 착수","용인시,용인,화성,구축사업,광역,버스,정보,구축,사업,착수,아시아투데이,홍화표,용인..."
